In [2]:
import tibber as t
import json

In [3]:
with open("config.json", "r") as f:
    TIBBER = json.load(f)

In [6]:
tibber = t.Tibber(access_token=TIBBER["TIBBER_API_KEY"])

In [11]:
tibber.get_home_ids()

[]